In [3]:
import pandas as pd
import gcsfs
import pandas_gbq
from google.cloud import bigquery
import numpy as np
import pickle

In [11]:
def scoring_model_A():
    client = bigquery.Client()
    table_id = 'Books.books_title_author'
    sql = "SELECT * FROM `sue-gcp-learning-env.sales_data.this_week`" 
    year_week_table = client.query(sql).to_dataframe()

    year_week_table['this_week_value'] = int(year_week_table['this_week_value'])
    year_week = year_week_table['this_week_value'].iloc[0]
    
    client = bigquery.Client()
    table_id = 'sales_data.superstore_sales_data_processed'
    sql = "SELECT * EXCEPT(total_sales) FROM `sue-gcp-learning-env.sales_data.superstore_sales_data_processed`  where segment = 'Consumer' and year_week =?" 

    job_config = bigquery.QueryJobConfig(
        query_parameters = [
            bigquery.ScalarQueryParameter(None, "INTEGER" , int(year_week))
        ]
    )

    sales_data = client.query(sql, job_config=job_config).to_dataframe()
    print("Data Set Loaded")
    
    data = sales_data.set_index(['order_date','segment','country', 'city', 'state','category', 'subcategory','ship_mode','year_week', 'day_of_week' ])
    data['week_number'] = data['week_number'].map(int)  
    
    pickle_file_name = 'model_A_consumer.pkl'

    with open(pickle_file_name,'rb') as pickle_model:
        model = pickle.load(pickle_model)
        
    forecast = pd.DataFrame(model.predict(data))
    
    predictions = forecast
    predictions['predict']= forecast
    predictions = predictions['predict']
    
    data = data.reset_index()[['order_date','segment','country', 'city', 'state','category', 'subcategory','ship_mode']]
    final_data = pd.merge(data, predictions , how = 'left', left_index=True , right_index=True)
    
    #Push table to Google Big Query

    client = bigquery.Client()
    project_id = 'sue-gcp-learning-env'
    table_id = 'sales_data.forecast_result'
    pandas_gbq.to_gbq(final_data, table_id, project_id=project_id, if_exists='append')
    print("Uploaded Forecast Result to Big Query Table")
        
    return ("Result has been Updated")
    

In [13]:
def scoring_model_B():
    client = bigquery.Client()
    table_id = 'Books.books_title_author'
    sql = "SELECT * FROM `sue-gcp-learning-env.sales_data.this_week`" 
    year_week_table = client.query(sql).to_dataframe()

    year_week_table['this_week_value'] = int(year_week_table['this_week_value'])
    year_week = year_week_table['this_week_value'].iloc[0]
    
    client = bigquery.Client()
    table_id = 'sales_data.superstore_sales_data_processed'
    sql = "SELECT * EXCEPT(total_sales) FROM `sue-gcp-learning-env.sales_data.superstore_sales_data_processed`  where segment = 'Corporate' and year_week =?" 

    job_config = bigquery.QueryJobConfig(
        query_parameters = [
            bigquery.ScalarQueryParameter(None, "INTEGER" , int(year_week))
        ]
    )

    sales_data = client.query(sql, job_config=job_config).to_dataframe()
    print("Data Set Loaded")
    
    data = sales_data.set_index(['order_date','segment','country', 'city', 'state','category', 'subcategory','ship_mode','year_week', 'day_of_week' ])
    data['week_number'] = data['week_number'].map(int)  
    
    pickle_file_name = 'model_B_corporate.pkl'

    with open(pickle_file_name,'rb') as pickle_model:
        model = pickle.load(pickle_model)
        
    forecast = pd.DataFrame(model.predict(data))
    
    predictions = forecast
    predictions['predict']= forecast
    predictions = predictions['predict']
    
    data = data.reset_index()[['order_date','segment','country', 'city', 'state','category', 'subcategory','ship_mode']]
    final_data = pd.merge(data, predictions , how = 'left', left_index=True , right_index=True)
    
    #Push table to Google Big Query

    client = bigquery.Client()
    project_id = 'sue-gcp-learning-env'
    table_id = 'sales_data.forecast_result'
    pandas_gbq.to_gbq(final_data, table_id, project_id=project_id, if_exists='append')
    print("Uploaded Forecast Result to Big Query Table")
        
    return ("Result has been Updated")
    

In [14]:
scoring_model_B()

Data Set Loaded


1it [00:06,  6.04s/it]

Uploaded Forecast Result to Big Query Table


'Result Completed for Model_B'